In [1]:
# ignore warnings
import warnings

warnings.filterwarnings("ignore")

from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml

# Load data

In [2]:
def load_mtpl2(n_samples=None):
    """Fetch the French Motor Third-Party Liability Claims dataset.

    Parameters
    ----------
    n_samples: int, default=None
      number of samples to select (for faster run time). Full dataset has
      678013 samples.
    """
    # freMTPL2freq dataset from https://www.openml.org/d/41214
    df_freq = fetch_openml(data_id=41214, as_frame=True).data
    df_freq["IDpol"] = df_freq["IDpol"].astype(int)
    df_freq.set_index("IDpol", inplace=True)

    # freMTPL2sev dataset from https://www.openml.org/d/41215
    df_sev = fetch_openml(data_id=41215, as_frame=True).data

    # sum ClaimAmount over identical IDs
    df_sev = df_sev.groupby("IDpol").sum()

    df = df_freq.join(df_sev, how="left")
    df["ClaimAmount"].fillna(0, inplace=True)

    # unquote string fields
    for column_name in df.columns[df.dtypes.values == object]:
        df[column_name] = df[column_name].str.strip("'")
    return df.iloc[:n_samples]


try:
    # load data from local pickle
    df = pd.read_pickle("mtpl2.pkl")
except FileNotFoundError:
    # or download from openml
    df = load_mtpl2()
    df.to_pickle("mtpl2.pkl")
df

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
IDpol,,,,,,,,,,,,
1,1,0.10000,D,5,0,55,50,B12,Regular,1217,R82,0.0
3,1,0.77000,D,5,0,55,50,B12,Regular,1217,R82,0.0
5,1,0.75000,B,6,2,52,50,B12,Diesel,54,R22,0.0
10,1,0.09000,B,7,0,46,50,B12,Diesel,76,R72,0.0
11,1,0.84000,B,7,0,46,50,B12,Diesel,76,R72,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6114326,0,0.00274,E,4,0,54,50,B12,Regular,3317,R93,0.0
6114327,0,0.00274,E,4,0,41,95,B12,Regular,9850,R11,0.0
6114328,0,0.00274,D,6,2,45,50,B12,Diesel,1323,R82,0.0


Feature names
- IDpol: policy number (unique identifier); 
- ClaimNb: number of claims on the given policy; 
- Exposure: total exposure in yearly units;
- Area: area code (categorical, ordinal);
- VehPower: power of the car (categorical, ordinal)
- VehAge: age of the car in years; 
- DrivAge: age of the (most common) driver in years
- BonusMalus: bonus-malus level between 50 and 230 (with reference level 100); 
- VehBrand: car brand (categorical, nominal); 
- VehGas: diesel or regular fuel car (binary); 
- Density: density of inhabitants per km2 in the city of the living place of the driver; 
- Region: regions in France (prior to 2016)
- ClaimAmount: total claim amount.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 678013 entries, 1 to 6114330
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   ClaimNb      678013 non-null  int64   
 1   Exposure     678013 non-null  float64 
 2   Area         678013 non-null  category
 3   VehPower     678013 non-null  int64   
 4   VehAge       678013 non-null  int64   
 5   DrivAge      678013 non-null  int64   
 6   BonusMalus   678013 non-null  int64   
 7   VehBrand     678013 non-null  category
 8   VehGas       678013 non-null  object  
 9   Density      678013 non-null  int64   
 10  Region       678013 non-null  category
 11  ClaimAmount  678013 non-null  float64 
dtypes: category(3), float64(2), int64(6), object(1)
memory usage: 53.7+ MB


In [4]:
df.describe()

,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Density,ClaimAmount
count,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,6.780130e+05
mean,0.053247,0.528750,6.454631,7.044265,45.499122,59.761502,1792.422405,8.835998e+01
std,0.240117,0.364442,2.050906,5.666232,14.137444,15.636658,3958.646564,5.822454e+03
min,0.000000,0.002732,4.000000,0.000000,18.000000,50.000000,1.000000,0.000000e+00
25%,0.000000,0.180000,5.000000,2.000000,34.000000,50.000000,92.000000,0.000000e+00
50%,0.000000,0.490000,6.000000,6.000000,44.000000,50.000000,393.000000,0.000000e+00
75%,0.000000,0.990000,7.000000,11.000000,55.000000,64.000000,1658.000000,0.000000e+00
max,16.000000,2.010000,15.000000,100.000000,100.000000,230.000000,27000.000000,4.075401e+06


# preprocess data

In [5]:
# Remove outliers
# Correct for unreasonable observations (that might be data error)
# and a few exceptionally large claim amounts
df["ClaimNb"] = df["ClaimNb"].clip(upper=4)
df["Exposure"] = df["Exposure"].clip(upper=1)
df["ClaimAmount"] = df["ClaimAmount"].clip(upper=200000)

# If the claim amount is 0, then we do not count it as a claim. The loss function
# used by the severity model needs strictly positive claim amounts. This way
# frequency and severity are more consistent with each other.
df.loc[(df["ClaimAmount"] == 0) & (df["ClaimNb"] >= 1), "ClaimNb"] = 0

df["Frequency"] = df["ClaimNb"] / df["Exposure"]
# df = df.drop(columns=["ClaimNb", "Exposure", "ClaimAmount"])
df

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,Frequency
IDpol,,,,,,,,,,,,,
1,0,0.10000,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0
3,0,0.77000,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0
5,0,0.75000,B,6,2,52,50,B12,Diesel,54,R22,0.0,0.0
10,0,0.09000,B,7,0,46,50,B12,Diesel,76,R72,0.0,0.0
11,0,0.84000,B,7,0,46,50,B12,Diesel,76,R72,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6114326,0,0.00274,E,4,0,54,50,B12,Regular,3317,R93,0.0,0.0
6114327,0,0.00274,E,4,0,41,95,B12,Regular,9850,R11,0.0,0.0
6114328,0,0.00274,D,6,2,45,50,B12,Diesel,1323,R82,0.0,0.0


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    KBinsDiscretizer,
    OneHotEncoder,
    StandardScaler,
)

log_scale_transformer = make_pipeline(
    FunctionTransformer(func=np.log), StandardScaler()
)

column_trans = ColumnTransformer(
    [
        (
            "BN",  # binned_numeric
            KBinsDiscretizer(n_bins=10, subsample=int(2e5), random_state=0),
            ["VehAge", "DrivAge"],
        ),
        (
            "OH",  # onehot_categorical
            OneHotEncoder(),
            ["VehBrand", "VehPower", "VehGas", "Region", "Area"],
        ),
        # ("passthrough_numeric", "passthrough", ["BonusMalus"]),
        # ("log_scaled_numeric", log_scale_transformer, ["Density"]),
    ],
    remainder="passthrough",
)

df = pd.DataFrame(column_trans.fit_transform(df).toarray(), columns=column_trans.get_feature_names_out())
df = df.rename(
    columns={_: _.replace("__", "_") if "remainder__" not in _ else _.replace("remainder__", "") for _ in df.columns})

standard_scaler = StandardScaler()
df["Density"] = standard_scaler.fit_transform(np.log(df["Density"]).values.reshape(-1, 1))

df

,BN_VehAge_0.0,BN_VehAge_1.0,BN_VehAge_2.0,BN_VehAge_3.0,BN_VehAge_4.0,BN_VehAge_5.0,BN_VehAge_6.0,BN_VehAge_7.0,BN_VehAge_8.0,BN_VehAge_9.0,...,OH_Area_C,OH_Area_D,OH_Area_E,OH_Area_F,ClaimNb,Exposure,BonusMalus,Density,ClaimAmount,Frequency
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.10000,50.0,0.600055,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.77000,50.0,0.600055,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.75000,50.0,-1.065404,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.09000,50.0,-0.882694,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.84000,50.0,-0.882694,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678008,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.00274,50.0,1.136113,0.0,0.0
678009,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.00274,95.0,1.718010,0.0,0.0
678010,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.00274,50.0,0.644703,0.0,0.0
678011,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00274,50.0,-0.763395,0.0,0.0


In [10]:
# split train test data
from sklearn.model_selection import train_test_split

X = df.drop(columns=["ClaimNb", "Exposure", "ClaimAmount", "Frequency"])
y = df[["ClaimNb", "Exposure", "ClaimAmount", "Frequency"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

y_train_exposure = y_train["Exposure"]
y_test_exposure = y_test["Exposure"]

y_train = y_train["Frequency"]
y_test = y_test["Frequency"]

In [47]:
def get_score(y_test, y_pred, weight):
    from sklearn.metrics import mean_squared_error, mean_poisson_deviance, mean_absolute_error, max_error
    if np.any(y_pred < 0):
        print("Negative predictions found, replacing with 0")
        y_pred = np.where(y_pred < 0, np.finfo(float).eps, y_pred)

    print(f"MSE: {mean_squared_error(y_test, y_pred, sample_weight=weight):.4f}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred, sample_weight=weight):.4f}")
    print(f"Max Error: {max_error(y_test, y_pred):.4f}")
    print(f"Mean Poisson Deviance: {mean_poisson_deviance(y_test, y_pred, sample_weight=weight):.4f}")

# Intercept Only Model

In [48]:

y_pred = y_train.mean() * np.ones_like(y_test)
get_score(y_test, y_pred, y_test_exposure)

MSE: 0.2268
MAE: 0.1817
Max Error: 364.8799
Mean Poisson Deviance: 0.4960


# Linear Regression

In [49]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train, sample_weight=y_train_exposure)
y_pred = lr.predict(X_test)
get_score(y_test, y_pred, y_test_exposure)

Negative predictions found, replacing with 0
MSE: 0.2227
MAE: 0.1378
Max Error: 364.8496
Mean Poisson Deviance: 0.4636


# GLM Poisson

In [50]:
from sklearn.linear_model import PoissonRegressor

pr = PoissonRegressor(
    alpha=1e-4, solver="newton-cholesky",
)
pr.fit(X_train, y_train, sample_weight=y_train_exposure)
y_pred = pr.predict(X_test)
get_score(y_test, y_pred, y_test_exposure)

MSE: 0.2237
MAE: 0.1376
Max Error: 364.8857
Mean Poisson Deviance: 0.4546


# XGBoost

In [58]:

# import multiprocessing

# from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import GradientBoostingRegressor

# param_grid = {"loss": ['absolute_error', 'quantile', 'squared_error', 'huber'], "n_estimators": [100, 1000, 2000]}
# 
# xgb_rscv = RandomizedSearchCV(
#     GradientBoostingRegressor(),
#     param_grid,
#     n_iter=35,
#     verbose=True,
#     cv=3,
#     n_jobs=multiprocessing.cpu_count() - 1,
#     scoring='neg_mean_squared_error',
#     return_train_score=True)
# 
# xgb_rscv.fit(X_train, y_train)
# #check top performing model parameters
# best_param = xgb_rscv.best_params_
# print(best_param)
# #check top performing model's score
# print(xgb_rscv.best_score_)

xgb = GradientBoostingRegressor(
    loss="huber",
    n_estimators=100,
    verbose=True,
)
xgb.fit(X_train, y_train, sample_weight=y_train_exposure)
y_pred = xgb.predict(X_test)
get_score(y_test, y_pred, y_test_exposure)


      Iter       Train Loss   Remaining Time 
         1           0.0000            5.50s
         2           0.0000            5.17s
         3           0.0000            5.14s
         4           0.0000            5.16s
         5           0.0000            5.20s
         6           0.0000            5.12s
         7           0.0000            5.06s
         8           0.0000            5.02s
         9           0.0000            4.94s
        10           0.0000            4.85s
        20           0.0000            4.25s
        30           0.0000            3.77s
        40           0.0000            3.23s
        50           0.0000            2.68s
        60           0.0000            2.14s
        70           0.0000            1.61s
        80           0.0000            1.07s
        90           0.0000            0.53s
       100           0.0000            0.00s
MSE: 0.2300
MAE: 0.0732
Max Error: 365.0000


ValueError: Mean Tweedie deviance error with power=1 can only be used on non-negative y and strictly positive y_pred.

In [52]:
get_score(y_test, y_pred, y_test_exposure)

MSE: 0.2300
MAE: 0.0732
Max Error: 365.0000


ValueError: Mean Tweedie deviance error with power=1 can only be used on non-negative y and strictly positive y_pred.